In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

In [3]:
# get data
# dataset = datasets.ML100k()
dataset = datasets.ML20m()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set(use_negative_sampling=True)

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)
#item_feature_set = torch.LongTensor(item_feature_set).to(device)

# set hyperparameters
lr = 1e-3
n_dim = 50

# set model
#distribution = distributions.Empirical()
#distribution = distributions.Gaussian()
#distribution = distributions.Gamma()
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
#model = models.MutualProximityCML(n_user, n_item, distribution, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# set evaluator
ks=[10, 50]
evaluator = evaluators.RecallEvaluator(test_set, ks)
#evaluator = evaluators.CoverageEvaluator(test_set, ks)
#evaluator = evaluators.DiversityEvaluator(test_set, item_feature_set, ks, emb_sim=False)
#evaluator = evaluators.HubnessEvaluator(test_set, ks)
#evaluator = evaluators.UnpopularityEvaluator(test_set, ks)
#evaluator = evaluators.F1ScoreEvaluator(test_set, ks)
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [4]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

 23%|█████████████▏                                            | 31527/138287 [00:57<03:15, 546.64it/s]


KeyboardInterrupt: 

In [ ]:
trainer.valid_scores

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)